# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [ ]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_smooth,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.189663628933333 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOBBIES_1_055_CA_1_validation,3.0,2.037486,0.1,0.1
1,HOBBIES_1_055_CA_1_validation,3.0,2.129914,0.1,0.1
2,HOBBIES_1_055_CA_1_validation,3.0,2.213826,0.1,0.1
3,HOBBIES_1_055_CA_1_validation,1.0,2.289935,0.1,0.1
4,HOBBIES_1_055_CA_1_validation,2.0,2.196358,0.1,0.1
...,...,...,...,...,...
122872,HOBBIES_1_354_CA_4_validation,0.0,0.000000,0.9,0.9
122873,HOBBIES_1_354_CA_4_validation,0.0,0.000000,0.9,0.9
122874,HOBBIES_1_354_CA_4_validation,0.0,0.000000,0.9,0.9
122875,HOBBIES_1_354_CA_4_validation,0.0,0.000000,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Smooth_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summary all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.09231365583333352 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_2_validation,0.1,0.1,0.730833,66.2491,76.9546,88.9783,2.882732,3.594212
1,HOBBIES_1_004_CA_2_validation,0.1,0.2,0.746614,67.6796,78.2611,90.9884,2.944976,3.644858
2,HOBBIES_1_004_CA_2_validation,0.1,0.3,0.756520,68.5776,79.3684,92.8243,2.984052,3.699659
3,HOBBIES_1_004_CA_2_validation,0.1,0.4,0.763146,69.1782,80.4054,94.5696,3.010187,3.760298
4,HOBBIES_1_004_CA_2_validation,0.1,0.5,0.771358,69.9226,81.7990,96.5745,3.042578,3.825784
...,...,...,...,...,...,...,...,...,...
3316,HOBBIES_1_354_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
3317,HOBBIES_1_354_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
3318,HOBBIES_1_354_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
3319,HOBBIES_1_354_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOBBIES_1_005_CA_4_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOBBIES_1_005_CA_4_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOBBIES_1_005_CA_4_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOBBIES_1_005_CA_4_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOBBIES_1_005_CA_4_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3316,HOBBIES_1_354_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3317,HOBBIES_1_354_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3318,HOBBIES_1_354_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3319,HOBBIES_1_354_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOBBIES_1_005_CA_4_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOBBIES_1_005_CA_4_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOBBIES_1_005_CA_4_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOBBIES_1_005_CA_4_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOBBIES_1_005_CA_4_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3316,HOBBIES_1_354_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3317,HOBBIES_1_354_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3318,HOBBIES_1_354_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3319,HOBBIES_1_354_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOBBIES_1_005_CA_4_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOBBIES_1_005_CA_4_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOBBIES_1_005_CA_4_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOBBIES_1_005_CA_4_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOBBIES_1_005_CA_4_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3316,HOBBIES_1_354_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3317,HOBBIES_1_354_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3318,HOBBIES_1_354_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3319,HOBBIES_1_354_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOBBIES_1_005_CA_4_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOBBIES_1_005_CA_4_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOBBIES_1_005_CA_4_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOBBIES_1_005_CA_4_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOBBIES_1_005_CA_4_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3316,HOBBIES_1_354_CA_4_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3317,HOBBIES_1_354_CA_4_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3318,HOBBIES_1_354_CA_4_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3319,HOBBIES_1_354_CA_4_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 9.75609756097561 %
Percentage of unexpected values of WMAPE is: 9.75609756097561 %
Percentage of unexpected values of SMAPE is: 12.195121951219512 %
Percentage of unexpected values of MAPE is: 9.75609756097561 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                
0.1   0.1   0.801077  68.303283   83.552617  56.705383  1.901061  2.433368
      0.2   0.798900  68.038764   84.031428  57.308822  1.904912  2.446461
      0.3   0.802442  68.359792   84.876442  58.404939  1.914798  2.467347
      0.4   0.807431  68.817786   85.924500  59.720831  1.928891  2.492568
      0.5   0.816718  69.696761   87.220322  61.548275  1.947985  2.521143
...              ...        ...         ...        ...       ...       ...
0.9   0.5   0.917798  78.081097   95.750044  76.429839  2.226728  2.910591
      0.6   0.926970  78.910378   97.079117  78.361919  2.255199  2.950466
      0.7   0.937441  79.839361   98.706150  80.443286  2.287843  2.994965
      0.8   0.950394  80.973997  100.939186  82.932622  2.326376  3.045241
      0.9   0.965607  82.321642  103.972161  85.845525  2.369576  3.102875

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE      SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                             
0.1   0.1   0.763672  66.33900   79.45860  54.23970  1.594910  1.997687
      0.2   0.761651  66.98195   79.82460  53.90710  1.629030  2.012525
      0.3   0.767126  67.24495   79.95970  55.56940  1.645636  2.035051
      0.4   0.771357  67.62870   80.63850  55.45420  1.655772  2.058927
      0.5   0.778163  68.32100   81.98620  56.46330  1.672449  2.080876
...              ...       ...        ...       ...       ...       ...
0.9   0.5   0.902750  79.73030   90.07565  71.99450  1.968266  2.500207
      0.6   0.916286  80.30615   91.45770  73.01245  1.958414  2.527015
      0.7   0.931051  81.83220   93.36000  74.94855  1.965179  2.549361
      0.8   0.944849  83.61010   96.32995  78.29520  1.992875  2.577795
      0.9   0.958868  85.40140  100.32255  81.75670  2.023441  2.614740

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.1)


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.1
best_beta = 0.2

In [ ]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_smooth, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.005822731249999909 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_2_validation,0.636727,71.9429,72.5832,106.6597,3.134654,3.878419
1,HOBBIES_1_004_CA_3_validation,0.703563,37.4487,41.8765,30.9276,2.327169,2.958515
2,HOBBIES_1_005_CA_4_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOBBIES_1_040_CA_3_validation,0.959062,86.0697,113.2813,43.3481,1.106610,1.432752
4,HOBBIES_1_048_CA_4_validation,0.649855,92.9480,93.2475,91.8026,4.249054,6.638637
5,HOBBIES_1_055_CA_1_validation,0.645301,89.6696,103.0201,65.5000,1.985541,2.609403
6,HOBBIES_1_055_CA_3_validation,0.913698,71.5623,90.5887,59.4168,1.686827,2.060562
7,HOBBIES_1_056_CA_3_validation,0.947096,78.6818,84.1806,72.9880,1.967046,2.186742
8,HOBBIES_1_064_CA_1_validation,0.668084,68.3502,82.8541,42.5390,0.976431,1.138641
9,HOBBIES_1_064_CA_3_validation,0.852372,61.8611,72.7893,29.5566,2.032579,2.958654


In [ ]:
df_result_final.to_csv('CrostonTSB_Smooth_Test_Data.csv')